In [70]:
import pandas as pd
import numpy as np
import re
import preprocessor as p
from langdetect import detect
from string import punctuation
from collections import Counter
import numpy as np
import tensorflow as tf
from string import punctuation
from collections import Counter
import collections
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [123]:
s='volvov602016'

dd=pd.read_csv(s+'.csv',sep=';')
dd.head()

,username,date,retweets,favorites,text,geo,mentions,hashtags,id,permalink
0,NaN,2016-12-31 23:15,0,0,I'm at Tom's Volvo V60 https://www. swarmapp.c...,NaN,NaN,NaN,815335550639017985,https://twitter.com/_TomMartens/status/8153355...
1,NaN,2016-12-31 18:57,0,0,Volvo V60 bjr 2013 Ex.BTW 0% Bijtelling!! 2.4 ...,NaN,NaN,NaN,815270789197025284,https://twitter.com/eenhuismanauto/status/8152...
2,NaN,2016-12-31 16:38,0,0,Volvo V60 1.6 T4 R-DESIGN. 84SZD9 163.430 km P...,NaN,NaN,NaN,815235783774924800,https://twitter.com/vanvlotenoccasi/status/815...
3,NaN,2016-12-31 15:13,0,0,Volvo V60 bjr 2014 2.0 D4 133kW/181pk 6-bak R-...,NaN,@YouTube,NaN,815214293117501442,https://twitter.com/eenhuismanauto/status/8152...
4,NaN,2016-12-31 15:11,0,0,Volvo V60 bjr 2014 2.0 D4 133kW/181pk 6-bak R-...,NaN,NaN,NaN,815213875050270724,https://twitter.com/eenhuismanauto/status/8152...


In [124]:


def cc(dl,ss):
    df=dl
    df['clean_text']=pd.Series()
    dg=pd.DataFrame()
    for i in range(len(df)):
        s=str(p.clean(df['text'][i]))
        df['clean_text'][i]=s
    df['clean_text']=df.clean_text.str.replace('http://','')
    df['clean_text']=df.clean_text.str.replace('https://','')
    df['clean_text']=df.clean_text.str.replace('$','')
    df['clean_text']=df.clean_text.str.replace('»','')
    df['clean_text']=df.clean_text.str.replace('-','')
    for c in punctuation:
        df['clean_text']=df.clean_text.str.replace(c,'')
    
    dg['clean_text']=pd.Series()
    dg['retweets']=pd.Series()
    dg['favorites']=pd.Series()
    for i in range(len(df)):
        if len(str(df['clean_text'][i])) > 20:
            try:
                x=detect(str(df['clean_text'][i]))
            except Exception:
                pass
        if  x== 'en':
            dg.loc[len(dg)]=[df['clean_text'][i],df['retweets'][i],df['favorites'][i]]
    temp=open(ss+'.txt','w')
    for i in range(len(dg)):
        if len(str(dg['clean_text'][i])) < 10:
            dg['clean_text'][i]=None
    dg.dropna(inplace=True)
    dg.index=pd.RangeIndex(len(dg.index))
    for i in range(len(dg)):
        temp.write(str(dg['clean_text'][i])+'\n')
    temp.close()
    return dg
ds=cc(dd,s)
ds.to_csv('temp.csv',index=None,sep='$')
print(ds.head(),'\n',len(ds),'\n',ds.tail())

                                          clean_text  retweets  favorites
0  New post Review The Volvo V60 T5 AWD Cross Cou...       0.0        0.0
1  Volvo V60 Cross Country D4 pk Summum Intellisa...       0.0        0.0
2  Road Test Review  Volvo V60 Cross Country Plat...       0.0        0.0
3  Ned and his son Grady taking delivery of their...       0.0        0.0
4  Volvo V60  The Volvo V60 is the automakers sma...       0.0        0.0 
 47 
                                            clean_text  retweets  favorites
42  Volvo V60 D3 pk Geartronic6 Ocean Race Busines...       0.0        0.0
43       Is Due For A Refresh   volvov60duerefresh …        0.0        0.0
44  A Look Into The New bhp  polestarutmcampaigncr...       0.0        2.0
45  This is what the Volvo V60 could look like if ...       0.0        0.0
46  The Volvo V60 DH5 Diesel PlugIn Hybrid Fully C...       0.0        0.0


In [125]:
#ds=pd.read_csv('temp.csv',sep='$')
print(len(ds))
with open(s+'.txt', 'r') as f:
    reviews = f.read()
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')
all_text = ''.join(reviews)
words = all_text.split()

47


In [126]:
pickle_in = open("vocab_to_int.pickle","rb")
vocab_to_int = pickle.load(pickle_in)
reviews_ints=[]
for each in reviews:
    reviews_ints.append([vocab_to_int.get(word,0) for word in each.split()])

In [127]:
reviews_ints = [r for r in reviews_ints if len(r) > 0]
print(len(reviews_ints),reviews_ints[0])
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))
seq_len = 30
features = np.zeros((len(reviews_ints), seq_len), dtype=int)
for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]
len(features)

47 [28, 124, 165, 19, 309, 0, 0, 224, 0, 0, 11, 3, 0, 0, 0, 43, 0, 0, 13, 201]
Zero-length reviews: 0
Maximum review length: 22


47

In [128]:
lstm_size = 25
lstm_layers = 1
batch_size = len(features)
learning_rate = 0.001
embed_size = 30 
n_words = len(vocab_to_int) #+ 1 # Add 1 for 0 added to vocab
print(n_words)

400


In [129]:
tf.reset_default_graph()
with tf.name_scope('inputs'):
    inputs_ = tf.placeholder(tf.int32, [None, 30], name="inputs")
    labels_ = tf.placeholder(tf.float32, [None,], name="labels")
    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
with tf.name_scope("Embeddings"):
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)
def lstm_cell():
    #lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size, reuse=tf.get_variable_scope().reuse)
    lstm=tf.contrib.rnn.BasicRNNCell(num_units=lstm_size,reuse=tf.get_variable_scope().reuse)
    return tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
with tf.name_scope("RNN_layers"):
    cell = tf.contrib.rnn.MultiRNNCell([lstm_cell() for _ in range(lstm_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32) 
    
with tf.name_scope("RNN_forward"):
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)  
    
with tf.name_scope('predictions'):
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=None)
    
with tf.name_scope('cost'):
    #cost = tf.losses.mean_squared_error(labels_, predictions)
    #cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_,logits=predictions))
    cost=tf.reduce_mean(tf.squared_difference(labels_,predictions))
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
with tf.name_scope('validation'):
    #correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    #accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    #correct = tf.nn.in_top_k(predictions,labels_ , 1)
    #accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    total_error = tf.reduce_sum(tf.square(labels_ - tf.reduce_mean(labels_)))
    unexplained_error = tf.reduce_sum(tf.square(labels_ - predictions))
    accuracy = 1- tf.div(total_error, unexplained_error)
    
tf.summary.histogram('predictions', predictions)
tf.summary.scalar('cost', cost)
merged = tf.summary.merge_all()

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "checkpoints/sentiment_final_RNN.ckpt")
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    feed = {inputs_: np.array(features),
            keep_prob: 1,
            initial_state: test_state}
    pred=predictions.eval(feed_dict=feed)

INFO:tensorflow:Restoring parameters from checkpoints/sentiment_final_RNN.ckpt


In [130]:
al=1
bet=1
ds['sento_anal']=pred.reshape(-1)
print(ds.head(),'\n',ds.tail())
ds.to_csv(s+'_final.csv',index=False,sep='$')

                                          clean_text  retweets  favorites  \
0  New post Review The Volvo V60 T5 AWD Cross Cou...       0.0        0.0   
1  Volvo V60 Cross Country D4 pk Summum Intellisa...       0.0        0.0   
2  Road Test Review  Volvo V60 Cross Country Plat...       0.0        0.0   
3  Ned and his son Grady taking delivery of their...       0.0        0.0   
4  Volvo V60  The Volvo V60 is the automakers sma...       0.0        0.0   

   sento_anal  
0    0.711845  
1    0.456815  
2    0.511756  
3    0.433515  
4    0.605128   
                                            clean_text  retweets  favorites  \
42  Volvo V60 D3 pk Geartronic6 Ocean Race Busines...       0.0        0.0   
43       Is Due For A Refresh   volvov60duerefresh …        0.0        0.0   
44  A Look Into The New bhp  polestarutmcampaigncr...       0.0        2.0   
45  This is what the Volvo V60 could look like if ...       0.0        0.0   
46  The Volvo V60 DH5 Diesel PlugIn Hybrid Fully 

In [131]:
ds.keys()

Index(['clean_text', 'retweets', 'favorites', 'sento_anal'], dtype='object')

In [104]:
len(pred)

57

In [15]:
len(ds)

88